In [10]:
import os
import random
from shutil import copyfile
from pydub import AudioSegment
import librosa
import pandas as pd
from maad import sound, util
from maad.rois import template_matching


def aggregate(rois_df, window=21):
    aggregated = []
    current_group = rois_df.iloc[0].copy()
    for index, row in rois_df.iterrows():
        if row['peak_time'] <= current_group['peak_time'] + window:
            current_group['xcorrcoef'] = max(current_group['xcorrcoef'], row['xcorrcoef'])
            current_group['max_t'] = max(current_group['max_t'], row['max_t'])
        else:
            aggregated.append(current_group)
            current_group = row.copy()
    aggregated.append(current_group)
    
    return pd.DataFrame(aggregated)

def get_timestamps(signal_name, template_name='pt16-m55013.wav', offset=0):
    flims = (6000, 12000)
    db_range = 80
    template, fs_template = librosa.load(template_name, sr=None)
    audio, fs_audio = librosa.load(signal_name, sr=None)

    Sxx_template, _, _, _ = sound.spectrogram(template, fs_template)
    Sxx_template = util.power2dB(Sxx_template, db_range)

    Sxx_audio, tn, fn, ext = sound.spectrogram(audio, fs_audio)
    Sxx_audio = util.power2dB(Sxx_audio, db_range)

    peak_th = 0.75
    xcorrcoef, rois = template_matching(Sxx_audio, Sxx_template, tn, ext, peak_th)
    rois['min_f'] = flims[0]
    rois['max_f'] = flims[1]
    rois_df = pd.DataFrame(rois)
    aggregated_rois = aggregate(rois_df)
    min_t_values = aggregated_rois['min_t'].values + offset
        
    if len(min_t_values) > 1:
        min_t_values = min_t_values[:-1]
    else:
        min_t_values = []
        
    return min_t_values

def process_audio_files(input_folder, output_folder, template_name='cutting/pt16-m55013.wav'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for root, dirs, files in os.walk(input_folder):
        for file_name in files:
            if file_name.endswith('.wav'):
                file_path = os.path.join(root, file_name)
                min_t_values = get_timestamps(file_path, template_name)
                
                if len(min_t_values) > 0:
                    audio = AudioSegment.from_wav(file_path)
                    for i, start_time in enumerate(min_t_values):
                        if i < len(min_t_values) - 1:
                            end_time = min_t_values[i+1] * 1000
                        else:
                            end_time = len(audio)
                        
                        start_time_ms = start_time * 1000
                        segment = audio[start_time_ms:end_time]
                        
                        # Extract label
                        category_label = os.path.basename(os.path.dirname(root))
                        
                        output_file_name = f"{category_label}_{file_name.split('.')[0]}_segment_{i+1}.wav"
                        output_path = os.path.join(output_folder, output_file_name)
                        segment.export(output_path, format="wav")
                        print(f"Segment {i+1} done in {output_file_name}")
                else:
                    print(f"Im File {file_name} keine Abschnitte gefunden!")




def split_files(input_folder, output_folder, splits=(0.7, 0.15, 0.15)):
    assert sum(splits) == 1.0, "Splits must sum to 1.0"
    
    for category in os.listdir(input_folder):
        category_path = os.path.join(input_folder, category)
        if os.path.isdir(category_path):
            files = [os.path.join(category_path, f) for f in os.listdir(category_path) if f.endswith('.wav')]
            random.shuffle(files)
            
            train_split = int(splits[0] * len(files))
            test_split = int(splits[1] * len(files))
            
            train_files = files[:train_split]
            test_files = files[train_split:train_split + test_split]
            val_files = files[train_split + test_split:]
            
            splits_map = {'train': train_files, 'test': test_files, 'val': val_files}
            
            for split, split_files in splits_map.items():
                split_folder = os.path.join(output_folder, category, split)
                if not os.path.exists(split_folder):
                    os.makedirs(split_folder)
                
                for file_path in split_files:
                    dest_path = os.path.join(split_folder, os.path.basename(file_path))
                    copyfile(file_path, dest_path)
                    print(f"Copied {file_path} to {dest_path}")


main_input_folder = 'ALL_audio'
temp_output_folder = 'temp_split_audio'
final_output_folder = 'ALL_segments'

split_files(main_input_folder, temp_output_folder)

for split in ['train', 'test', 'val']:
    for subfolder in ['alu_ohne_io', 'alu_ohne_nio', 'alu_nut_io', 'alu_nut_nio', 'kunststoff_nut_io', 'kunststoff_nut_nio', 'kunststoff_ohne_io', 'kunststoff_ohne_nio']:
        split_folder_path = os.path.join(temp_output_folder, subfolder, split)
        output_subfolder_path = os.path.join(final_output_folder, subfolder, split)
        process_audio_files(split_folder_path, output_subfolder_path)


Copied audio/FULL\alu_nut_io\URecorder_20240715_145135.wav to temp_split_audio\alu_nut_io\train\URecorder_20240715_145135.wav
Copied audio/FULL\alu_nut_io\URecorder_20240716_101108.wav to temp_split_audio\alu_nut_io\train\URecorder_20240716_101108.wav
Copied audio/FULL\alu_nut_io\URecorder_20240715_142039.wav to temp_split_audio\alu_nut_io\train\URecorder_20240715_142039.wav
Copied audio/FULL\alu_nut_io\URecorder_20240716_080440.wav to temp_split_audio\alu_nut_io\train\URecorder_20240716_080440.wav
Copied audio/FULL\alu_nut_io\URecorder_20240716_104921.wav to temp_split_audio\alu_nut_io\train\URecorder_20240716_104921.wav
Copied audio/FULL\alu_nut_io\URecorder_20240716_104104.wav to temp_split_audio\alu_nut_io\train\URecorder_20240716_104104.wav
Copied audio/FULL\alu_nut_io\URecorder_20240716_101940.wav to temp_split_audio\alu_nut_io\train\URecorder_20240716_101940.wav
Copied audio/FULL\alu_nut_io\URecorder_20240716_083244.wav to temp_split_audio\alu_nut_io\train\URecorder_20240716_083

C:\Users\Alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\maad\util\miscellaneous.py:413: RuntimeWarning: divide by zero encountered in log10
  y = 10*log10(x)   # take log


Segment 1 done in alu_ohne_io_URecorder_20240715_094020_segment_1.wav
Segment 2 done in alu_ohne_io_URecorder_20240715_094020_segment_2.wav
Segment 3 done in alu_ohne_io_URecorder_20240715_094020_segment_3.wav
Segment 4 done in alu_ohne_io_URecorder_20240715_094020_segment_4.wav
Segment 5 done in alu_ohne_io_URecorder_20240715_094020_segment_5.wav
Segment 6 done in alu_ohne_io_URecorder_20240715_094020_segment_6.wav
Segment 7 done in alu_ohne_io_URecorder_20240715_094020_segment_7.wav
Segment 8 done in alu_ohne_io_URecorder_20240715_094020_segment_8.wav
Segment 9 done in alu_ohne_io_URecorder_20240715_094020_segment_9.wav
Segment 10 done in alu_ohne_io_URecorder_20240715_094020_segment_10.wav
Segment 11 done in alu_ohne_io_URecorder_20240715_094020_segment_11.wav
Segment 12 done in alu_ohne_io_URecorder_20240715_094020_segment_12.wav
Segment 13 done in alu_ohne_io_URecorder_20240715_094020_segment_13.wav
Segment 14 done in alu_ohne_io_URecorder_20240715_094020_segment_14.wav
Segment 15

C:\Users\Alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\maad\util\miscellaneous.py:413: RuntimeWarning: divide by zero encountered in log10
  y = 10*log10(x)   # take log


Segment 1 done in alu_ohne_io_URecorder_20240715_095928_segment_1.wav
Segment 2 done in alu_ohne_io_URecorder_20240715_095928_segment_2.wav
Segment 3 done in alu_ohne_io_URecorder_20240715_095928_segment_3.wav
Segment 4 done in alu_ohne_io_URecorder_20240715_095928_segment_4.wav
Segment 5 done in alu_ohne_io_URecorder_20240715_095928_segment_5.wav
Segment 6 done in alu_ohne_io_URecorder_20240715_095928_segment_6.wav
Segment 7 done in alu_ohne_io_URecorder_20240715_095928_segment_7.wav
Segment 8 done in alu_ohne_io_URecorder_20240715_095928_segment_8.wav
Segment 9 done in alu_ohne_io_URecorder_20240715_095928_segment_9.wav
Segment 10 done in alu_ohne_io_URecorder_20240715_095928_segment_10.wav
Segment 11 done in alu_ohne_io_URecorder_20240715_095928_segment_11.wav
Segment 12 done in alu_ohne_io_URecorder_20240715_095928_segment_12.wav
Segment 13 done in alu_ohne_io_URecorder_20240715_095928_segment_13.wav
Segment 14 done in alu_ohne_io_URecorder_20240715_095928_segment_14.wav
Segment 15

...

Segment 1 done in alu_nut_nio_URecorder_20240717_130518_segment_1.wav
Segment 2 done in alu_nut_nio_URecorder_20240717_130518_segment_2.wav
Segment 3 done in alu_nut_nio_URecorder_20240717_130518_segment_3.wav
Segment 4 done in alu_nut_nio_URecorder_20240717_130518_segment_4.wav
Segment 5 done in alu_nut_nio_URecorder_20240717_130518_segment_5.wav
Segment 6 done in alu_nut_nio_URecorder_20240717_130518_segment_6.wav
Segment 7 done in alu_nut_nio_URecorder_20240717_130518_segment_7.wav
Segment 8 done in alu_nut_nio_URecorder_20240717_130518_segment_8.wav
Segment 9 done in alu_nut_nio_URecorder_20240717_130518_segment_9.wav
Segment 10 done in alu_nut_nio_URecorder_20240717_130518_segment_10.wav
Segment 11 done in alu_nut_nio_URecorder_20240717_130518_segment_11.wav
Segment 12 done in alu_nut_nio_URecorder_20240717_130518_segment_12.wav
Segment 13 done in alu_nut_nio_URecorder_20240717_130518_segment_13.wav
Segment 14 done in alu_nut_nio_URecorder_20240717_130518_segment_14.wav
Segment 15

C:\Users\Alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\maad\util\miscellaneous.py:413: RuntimeWarning: divide by zero encountered in log10
  y = 10*log10(x)   # take log


Segment 1 done in alu_nut_nio_URecorder_20240717_142102_segment_1.wav
Segment 2 done in alu_nut_nio_URecorder_20240717_142102_segment_2.wav
Segment 3 done in alu_nut_nio_URecorder_20240717_142102_segment_3.wav
Segment 4 done in alu_nut_nio_URecorder_20240717_142102_segment_4.wav
Segment 5 done in alu_nut_nio_URecorder_20240717_142102_segment_5.wav
Segment 6 done in alu_nut_nio_URecorder_20240717_142102_segment_6.wav
Segment 7 done in alu_nut_nio_URecorder_20240717_142102_segment_7.wav
Segment 8 done in alu_nut_nio_URecorder_20240717_142102_segment_8.wav
Segment 9 done in alu_nut_nio_URecorder_20240717_142102_segment_9.wav
Segment 10 done in alu_nut_nio_URecorder_20240717_142102_segment_10.wav
Segment 11 done in alu_nut_nio_URecorder_20240717_142102_segment_11.wav
Segment 12 done in alu_nut_nio_URecorder_20240717_142102_segment_12.wav
Segment 13 done in alu_nut_nio_URecorder_20240717_142102_segment_13.wav
Segment 14 done in alu_nut_nio_URecorder_20240717_142102_segment_14.wav
Segment 15

C:\Users\Alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\maad\util\miscellaneous.py:413: RuntimeWarning: divide by zero encountered in log10
  y = 10*log10(x)   # take log


Segment 1 done in alu_nut_nio_URecorder_20240717_143736_segment_1.wav
Segment 2 done in alu_nut_nio_URecorder_20240717_143736_segment_2.wav
Segment 3 done in alu_nut_nio_URecorder_20240717_143736_segment_3.wav
Segment 4 done in alu_nut_nio_URecorder_20240717_143736_segment_4.wav
Segment 5 done in alu_nut_nio_URecorder_20240717_143736_segment_5.wav
Segment 6 done in alu_nut_nio_URecorder_20240717_143736_segment_6.wav
Segment 7 done in alu_nut_nio_URecorder_20240717_143736_segment_7.wav
Segment 8 done in alu_nut_nio_URecorder_20240717_143736_segment_8.wav
Segment 9 done in alu_nut_nio_URecorder_20240717_143736_segment_9.wav
Segment 10 done in alu_nut_nio_URecorder_20240717_143736_segment_10.wav
Segment 11 done in alu_nut_nio_URecorder_20240717_143736_segment_11.wav
Segment 12 done in alu_nut_nio_URecorder_20240717_143736_segment_12.wav
Segment 13 done in alu_nut_nio_URecorder_20240717_143736_segment_13.wav
Segment 14 done in alu_nut_nio_URecorder_20240717_143736_segment_14.wav
Segment 15